In [1]:
from metaflow import Flow,get_metadata
print("Metadata",get_metadata())
from metaflow_train import FinalData
import chart_studio.plotly as py
import plotly.graph_objects  as go
import plotly.express as ps

Metadata local@/home/valay/Documents/Code-Workspace/ASU-Projects/Robotics/Pyrep-Testing


In [2]:
run = list(Flow('TrainingSimulatorFlow').runs('mutant_vs_simplerelu'))[0]

In [3]:
print("Simple 3 Layer ReLu vs Mutant")
for data in run.data.final_data:
    print(data)

Simple 3 Layer ReLu vs Mutant

        Agent Name : SimpleImmitationLearningAgent

        Total Training Data Size : 21073

        Model Arguements : {}

        Collected Gradients : No

        Simulation Results 

            Total Number of Episodes : 200

            Steps Per Episode : 50

            Number Of Converged Cases : 128

            %  Cases That Converged : 64.0
        

        Agent Name : ImmitationLearningMutantAgent

        Total Training Data Size : 21073

        Model Arguements : {}

        Collected Gradients : No

        Simulation Results 

            Total Number of Episodes : 200

            Steps Per Episode : 50

            Number Of Converged Cases : 47

            %  Cases That Converged : 23.5
        


In [4]:
print("Simple 6 Layer ReLu vs Mutant")
run_2 = list(Flow('TrainingSimulatorFlow').runs('mutant_vs_simple'))[0]
for data in run_2.data.final_data:
    print(data)

Simple 6 Layer ReLu vs Mutant

        Agent Name : SimpleImmitationLearningAgent

        Total Training Data Size : 21073

        Model Arguements : {}

        Collected Gradients : No

        Simulation Results 

            Total Number of Episodes : 200

            Steps Per Episode : 50

            Number Of Converged Cases : 61

            %  Cases That Converged : 30.5
        

        Agent Name : ImmitationLearningMutantAgent

        Total Training Data Size : 21073

        Model Arguements : {}

        Collected Gradients : No

        Simulation Results 

            Total Number of Episodes : 200

            Steps Per Episode : 50

            Number Of Converged Cases : 97

            %  Cases That Converged : 48.5
        


In [5]:
# smart_vs_mutant
print("Simple N Layer ReLu vs Mutant With Lesser Layers")
run_2 = list(Flow('TrainingSimulatorFlow').runs('smart_vs_mutant'))[0]
print(run_2)
for data in run_2.data.final_data:
    print(data)

Simple N Layer ReLu vs Mutant With Lesser Layers
Run('TrainingSimulatorFlow/1586252587257348')

        Agent Name : SimpleImmitationLearningAgent__4

        Total Training Data Size : 21073

        Model Arguements : {}

        Collected Gradients : No

        Simulation Results 

            Total Number of Episodes : 200

            Steps Per Episode : 100

            Number Of Converged Cases : 114

            %  Cases That Converged : 56.99999999999999
        

        Agent Name : SimpleImmitationLearningAgent__1

        Total Training Data Size : 21073

        Model Arguements : {}

        Collected Gradients : No

        Simulation Results 

            Total Number of Episodes : 200

            Steps Per Episode : 100

            Number Of Converged Cases : 108

            %  Cases That Converged : 54.0
        

        Agent Name : SimpleImmitationLearningAgent__2

        Total Training Data Size : 21073

        Model Arguements : {}

        Collected Gradi

In [6]:
mydateparser = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S +0000")
loss_plot = go.Figure(layout=dict(title=dict(text="Plot of Running Losses of All Models")))
legend = []
for i in range(len(run_2.data.final_data)):
    loss_op = run_2.data.final_data[i].loss
    agent_name = run_2.data.final_data[i].agent_name
    epochs = [j+1 for j in range(len(loss_op))]
    loss_plot.add_trace(go.Scatter(
                x=epochs,
                y=loss_op,
                name=agent_name,
                opacity=0.8))

loss_plot.update_layout(xaxis_title="Epochs",
    yaxis_title="Loss")
loss_plot.show()

In [7]:
import itertools
def get_key_map(arr):
    finalmap = []
    for i in itertools.product(*arr):
        finalmap.append(i)
    return finalmap


In [8]:
from plotly.subplots import make_subplots
import math
import datetime
def plot_grad_figures(final_data_arr):
    if not hasattr(final_data_arr[0],'gradients'): # Gradients are collected for whole Flow. So if 1st doesnt have. No one has. 
            return None
    if len(final_data_arr[0].gradients['avg']) == 0:
        return None 
    rows = math.ceil(len(final_data_arr)/2)
    last_time = datetime.datetime.now()
    index_queue = get_key_map([[i+1 for i in range(rows)],[1,2]]) # Make rows and columns. 
    fig = make_subplots(rows=rows, cols=2, start_cell="bottom-left",subplot_titles=[data.agent_name for data in final_data_arr])
    for data in final_data_arr:
        gradients = data.gradients
        row,col = index_queue.pop(0)
        
        for i in range(len(gradients['avg'])):
            avg_grads = gradients['avg'][i]
            layers = gradients['layer'][i]
            fig.add_trace(go.Scatter(
                        x=layers, \
                        y=avg_grads, \
                        line=dict(color='blue',width=1),
                        opacity=0.8),row=row,col=col)

            curr_time = datetime.datetime.now()
            if (curr_time - last_time).total_seconds() > 30:
                print("Completed %d Percentage of injesting to Chart "% ((i/len(agent.gradients['avg']))*100))
                last_time = curr_time
        
        fig.update_xaxes(tickmode ='array',tickvals = layers,ticktext = layers,tickangle=90,range=[0, len(avg_grads)],row=row,col=col)
        fig.update_layout(title_text="Gradient Flows of Different Agents",height=2000,showlegend=False,width=2000)

    return fig

def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass

In [9]:
def make_loss_plots(final_data_arr):
    rows = math.ceil(len(final_data_arr)/2)+1
    last_time = datetime.datetime.now()
    index_queue = get_key_map([[i+1 for i in range(rows)],[1,2]]) # Make rows and columns. 
    last_row,last_col = index_queue[-1]
    loss_plot = make_subplots(
        rows=rows,\
        cols=2,\
        subplot_titles=[data.agent_name for data in final_data_arr]+["All Losses In One Plot"], \
        specs=[ [{}, {}] for _ in range(rows-1) ]+ [[ {"colspan": 2}, None] ]
    )
    for data in final_data_arr:
        row,col = index_queue.pop(0)
        loss_op = data.loss
        if not isinstance(loss_op,list):
            continue
        agent_name = data.agent_name
        epochs = [j+1 for j in range(len(loss_op))]
        loss_plot.add_trace(go.Scatter(
                    x=epochs,
                    y=loss_op,
                    name=agent_name,
                    line=dict(width=1),
                    opacity=0.8),row=row,col=col)

        loss_plot.add_trace(go.Scatter(
                    x=epochs,
                    y=loss_op,
                    name=agent_name,
                    line=dict(width=1),
                    opacity=0.8),row=last_row,col=1)

        loss_plot.update_yaxes(title_text="Loss",row=row,col=col)
        loss_plot.update_xaxes(title_text="Epochs",row=row,col=col)
    
    loss_plot.update_yaxes(title_text="Loss",row=last_row,col=1)
    loss_plot.update_xaxes(title_text="Epochs",row=last_row,col=1)
    loss_plot.update_layout(title_text="Plot of Running Losses of All Models",height=2000,showlegend=True,width=2000)
    return loss_plot

In [10]:
def CountFrequency(my_list):       
    # Creating an empty dictionary  
    freq = {} 
    for items in my_list: 
        freq[items] = my_list.count(items) 
    x ,y=[],[]
    for key, value in freq.items(): 
        x.append(key)
        y.append(value)
    return x,y

In [11]:
def make_convergence_plots(final_data_arr):
    # Todo : Add Validation to check if there is no data. 
    rows = math.ceil(len(final_data_arr)/2)+1
    last_time = datetime.datetime.now()
    index_queue = get_key_map([[i+1 for i in range(rows)],[1,2]]) # Make rows and columns. 
    last_row,_ = index_queue[-1]
    convergence_plot = make_subplots(
        rows=rows,\
        cols=2,\
        subplot_titles=[data.agent_name for data in final_data_arr]+["Convergence Comparisons"], \
        specs=[ [{}, {}] for _ in range(rows-1) ]+ [[ {"colspan": 2}, None] ]
    )
    agent_names = []
    num_convergences = []
    for data in final_data_arr:
        row,col = index_queue.pop(0)
        simulation_analytics = data.simulation_analytics
        if not isinstance(simulation_analytics,dict):
            continue
        agent_names.append(data.agent_name)
        steps_to_convergence_of_agent = list(map(lambda x: x['steps_to_convergence'],simulation_analytics['convergence_metrics']))
        step_to_convergence,frequency_of_occurence = CountFrequency(steps_to_convergence_of_agent)
        num_convergences.append(len(simulation_analytics['convergence_metrics']))
        convergence_plot.add_trace(go.Scatter(
                    x=step_to_convergence,
                    y=frequency_of_occurence,
                    mode="markers",
                    opacity=0.8),row=row,col=col)

        convergence_plot.update_yaxes(title_text="Frequency",row=row,col=col)
        convergence_plot.update_xaxes(title_text="Num Steps",row=row,col=col)
    
    convergence_plot.add_trace(go.Bar(x=agent_names,y=num_convergences),row=last_row,col=1)
    convergence_plot.update_yaxes(title_text="Number Of Convergences",row=last_row,col=1)
    convergence_plot.update_xaxes(title_text="Models Name",row=last_row,col=1)
    convergence_plot.update_layout(title_text="Convergence Distributions For Different Agents",height=2000,showlegend=False,width=2000)
    return convergence_plot

In [12]:
run = list(Flow('TrainingSimulatorFlow').runs('smart_vs_mutant'))[0]

In [13]:
def save_main_run_data(run):
    run_dir_path = "RunAnalytics/Run-"+run.id
    safe_mkdir(run_dir_path)
    f = plot_grad_figures(run.data.final_data)
    if f:
        f.write_image(run_dir_path+"/model_gradients.png")
    
    write_file = open(run_dir_path+"/model_results.txt",'w')
    l = make_loss_plots(run.data.final_data)
    l.write_image(run_dir_path+"/model_losses.png")
    c = make_convergence_plots(run.data.final_data)
    c.write_image(run_dir_path+"/model_convergence.png")
    write_file.writelines('\n'.join([str(data) for data in run.data.final_data]))
    write_file.close()

In [14]:
for run in Flow('TrainingSimulatorFlow').runs():
    flow_init_datum = list(run.steps())[-1].task.data
    if flow_init_datum.num_demos >= 100 and flow_init_datum.num_epochs >= 100:
        save_main_run_data(run)